In [5]:
from pathlib import Path
import numpy as np
from PIL import Image
import glob
from facenet_models import FacenetModel
from vision_profile import FaceDatabase
from camera import take_picture
from estimation import detect_face_prob, extract_descriptors, plot_histograms, cosine_distance, is_true_positive


%matplotlib notebook


model = FacenetModel()

In [1]:
db = FaceDatabase(file_path="database.pkl")
for filename in glob.glob(r'/pics/train/*.jpg'):
    print(filename)
    pic = Image.open(filename)
    pic_arr = np.array(pic)
    boxes, probabilities, landmarks = model.detect(pic_arr)
    name = filename[11:-6]
    descriptors = model.compute_descriptors(pic_arr, boxes) #gets descriptor for every file
    db.add_image(name, descriptors)
db.save()
 
#pic = Image.open(r'pics/test/Will_Smith_test.jpg')  
#pic_arr = np.array(pic)

NameError: name 'FaceDatabase' is not defined

In [ ]:
match_distances = []
non_match_distances = []
true_positives = []
false_negatives = []

for filename in glob.glob(r'./pics/train/*.jpg'):

    pic = Image.open(filename)
    pic_arr = np.array(pic)
    boxes, probs, landmarks = model.detect(pic_arr)
    descriptors = model.compute_descriptors(pic_arr, boxes)
    # print(descriptors)

    # detections = [detect_face_prob(img, model) for img in pic_arr]
    # descriptors = [extract_descriptors(img, boxes, model) for img, (boxes, _, _) in zip(pic_arr, detections)]
    # descriptors = np.concatenate(descriptors, axis=0)
    # labels = np.concatenate([[label] * len(boxes) for label, (boxes, _, _) in zip(pic_arr, detections)], axis=0)

    # match_distances, non_match_distances = cosine_distance(descriptors, labels, model)
    # true_positives = [prob for _, probs in detections for prob in probs if is_true_positive(prob, labels)]
    # false_negatives = [prob for _, probs in detections for prob in probs if not is_true_positive(prob, labels)]

    name = filename.split('/')[-1].split('.')[0]
    labels = [name] * len(descriptors)

    for prob in probs:
        if is_true_positive(prob, detection_threshold=0.5): # 0.5 is placeholder
            true_positives.append(prob)
        else:
            false_negatives.append(prob)

    for i, descriptor1 in enumerate(descriptors):
        for j, descriptor2 in enumerate(descriptors):
            if i < j:
                distance = cosine_distance([descriptor1], [descriptor2])[0][0]
                if labels[i] == labels[j]:
                    match_distances.append(distance)
                else:
                    non_match_distances.append(distance)

plot_histograms(true_positives, false_negatives, match_distances, non_match_distances)